In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.__version__

'2.9.1'

In [3]:
keras.__version__

'2.9.0'

# Perceptron

El "perceptrón" es una de las arquitecturas de RNA más sencillas, creada en 1957 por Frank Rosenblatt. Se basa en una neurona artificial ligeramente diferente llamada Unidad Lógica Umbral (ULU).

Las entradas y salidas son números (en lugar de activadores binarios) y cada conexión de entrada está asociada a un peso.

La ULU cálcula una suma ponderada de sus entrada ($ x = w_1 x_1 + w_2 x_2 + ... + w_n x_n = X^T W $) y luego aplica una función escalonada a esa suma para producir el resultado (output): $ h_w (X) = step(z) $, donde $ z = X^T W $ 

<img src="https://quantstartmedia.s3.amazonaws.com/images/article-images/articles/introduction-to-artificial-neural-networks-and-the-perceptron/perceptron.png"  width="500" height="500">

La función escalonada que se utiliza frecuentemente en los perceptrones es la función escalonada Heaviside

$ Heavise(z) = $ { $ 0 $ si $ z < 0 $ ; $ 1 $ si $ z \geq 0 $

Un perceptrón consta de una sola capa ULU. Cuando todas las neuronas de una capa estan conectadas a todas las neuronas de la capa anterior (neuronas de entrada), estamos ante una capa completamente conectada o densa.

Se suele añadir una capa de sesgo adicional ($ x_0 = 1 $): representada por una neurona de sesgo que produce 1 todo el tiempo.

La siguiente imagen representa un perceptróm con dos entradas y tres salidas, este perceptrón puede clasificar simultaneamente tres clases binarias diferentes, siendo un clasificador multisalida.

<img src="https://www.danli.org/2021/06/21/hands-on-machine-learning-keras/chapters/10/10.5.png"  width="500" height="500">

Se puede cálcular con eficiencia la salida de una capa de neurona para varias instancias:

$ h_{W, b} (X) = \phi (XW + b) $

En la ecuación tenemos:

- $X$: Matriz de características de entrada. Una fila por instancia y una columna por característica.
- $W$: Matriz de pesos de conexión, excepto la neurona de sesgo. Tiene una fila por neurona de entrada y una columna por neurona artificial en la capa.
- $b$: Vector de sesgo, que contiene todos los pesos de conexión entre la neurona de sesgo y las neuronas artificiales.
- $\phi$: Función de activación: cuando las neuronas artificiales son ULU, es una función escalonada pero hay otras.

## ¿Cómo se entrena un perceptrón?

El algoritmo de entrenamiento del perceptrón se basa en la regla de Hebb de su libro de 1949, Organización de la Conducta (Debate), donde se sugiere que cuando una neurona biologica activa a otra, con frecuencia la conexión entre estas dos se fortalece. 

Los perceptrones se entrenan con una variación de esta regla que tiene en cuenta el error cometido por la red cuando hace una predicción. El perceptrón recibe las instancias de entrenamiento y realiza sus predicciones para cada instancia, Por cada neurona de salida que produce una predicción erronea, refueza los pesos de conexión a partir de las entradas que habrían contribuido a la predicción correcta. La regla de aprendizaje es la siguiente:

$ w_{i,j}^{Siguiente} =  w_{i,j} + \eta(y_j - \hat{y}_j) x_i $

- $w_{i, j}$: El peso de conexión entre la i-esima neurona de entrada y la j-esima neurona de salida.
- $x_i$: El valor de la i-esima neurona de salida para la instancia de entrenamiento actual.
- $\hat{y}_j$: El valor de salida de la j-esima neurona para la instancia de entrenamiento actual.
- $y_i$: El valor objetivo de la j-esima neurona para la instancia de entrenamiento actual.
- $\eta$: Tasa de aprendizaje.

El límite de decisión de cada neurona es lineal, así que los perceptrones son incapaces de aprender patrones complejos. Sin embargo, las instancias de entrenamiento son separables linealmente, lo que permite converger a uan solución.

La instancia en Scikit-Learn orfdere una solución para el perceptrón.

In [18]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [20]:
iris = load_iris()
X = iris.data[:, (2,3)] # longitud de pétalo, anchura del pétalo
y = (iris.target == 0).astype(np.int32) # ¿Iris setosa?

In [21]:
clf = Perceptron()
clf.fit(X, y)

Perceptron()

In [24]:
y_pred = clf.predict([[2, 0.5]])
y_pred

array([0], dtype=int32)

Los perceptrones no permiten obtener probabilidades, sino que hacen predicciones basándose en un umbral duro, por eso es deseable una regresión logistica.

# Perceptrón Multicapa (PMC) y la retropropagación

Un PMC consta de una capa de entrada (transito), una o varias capas de ULU (llamada capas ocultas) y una última capa de ULU (capa de salida).Las capas cercanas a la entrada suelen llamarse "capas inferiores", y las próximas a la salida "capas superiores". Cada capa, excepto la de salida, incluye una neurona de sesgo y está completamente conectada a la siguiente capa.

<img src="https://www.danli.org/2021/06/21/hands-on-machine-learning-keras/chapters/10/10.9.png"  width="500" height="500">

Cuando una RNA contiene una pila profunda de capas ocultas, se habla de una Red Neuronal Profunda (RNP). En el campo del Deep Learning se estudia RNA y, en general, modelos que contienen pilas profundas de cálculos. 

## Retropropagación

Es un método de entrenamiento de los PMC, propuesto en 1986, que introduce la retropropagación, similar a las técnicas de gradiente descendiente: en solo dos pasos (hacía delante y atrás) la red puede cálcular el gradiente de error de la red respecto a cada parámetro del modelo. En otras palabras, puede averiguar cómo habría que ajustar cada peso de conexión y cada término de sesgo para reducir el error.

# Extracción de datos Fashion MNIST

In [5]:
# Conjunto de datos similar a MNIST, con 70.000 imagenes en escala de grises de 28x28 pixels y 10 clases de articulos de moda
fashion_mnist = keras.datasets.fashion_mnist

In [9]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [10]:
X_train_full.shape

(60000, 28, 28)

In [11]:
X_train_full.dtype

dtype('uint8')

In [12]:
# Creando conjunto de validación
# Se escala la densidad de pixeles hasta el rango 0-1, dividiendo entre 255
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [13]:
# Se asocia las etiquetas númericas con nombres representativos
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [15]:
class_names[y_train[0]]

'Coat'

# Creación del modelo con la API Secuencial

Crea el modelo Sequential. es el tipo más sencillo de modelos en Keras para RNA. 

Consta de una sola única pila de capas conectadas secuencialmente.

In [ ]:
model = keras.models.Sequential()

Se construye la primera capa y se añade al modelo. Capa Flatten convierte la entrada de imagen en una matriz 1D. 

Esta capa no tiene parametros, solo existe para el preprocesamiento.

Dado que es la primera capa, se debe especificar el input_shape, que da la forma de la instancia.

In [ ]:
model.add(keras.layers.Flatten(input_shape[28, 28]))

Se añade una capa Dense oculta con 300 neuronas, que utiliza la función de activación ReLU. 

Cada capa Dense administra su propia matriz de pesos, que contiene los pesos de conexión entre las neuronas y sus salidas.

También gestiona un vector de términos de sesgo (uno por neurona), cuando recibe datos de entrada, computa la siguiente ecuación:

$ h_{W,b}(X) = \Phi(X W + b) $ 

In [ ]:
model.add(keras.layers.Dense(300, activation = 'relu'))

In [ ]:
model.add(keras.layers.Dense(100, activation = 'relu'))

In [ ]:
model.add(keras.layers.Dense(10, activation = 'softmax'))